In [6]:
#! pip install dedupe

In [22]:
# Using true_id...works fine as expected
import pandas as pd
from difflib import SequenceMatcher

# Read the CSV file
df = pd.read_csv(r'c:\Users\Dave Sisk\Repos\soft-relate-data\sample-data-messy_200.csv')

print(f"Original rows: {len(df)}")

# Function to calculate similarity between two strings
def string_similarity(a, b):
    if pd.isna(a) or pd.isna(b):
        return 0
    return SequenceMatcher(None, str(a).lower().strip(), str(b).lower().strip()).ratio()

# Find potential duplicates by comparing names within each true_id group
duplicate_clusters = []

for true_id in df['true_id'].unique():
    group = df[df['true_id'] == true_id].copy()
    if len(group) > 1:
        # Compare names within this true_id group
        cluster = {
            'true_id': true_id,
            'count': len(group),
            'records': []
        }
        for idx, row in group.iterrows():
            cluster['records'].append({
                'row_id': row['row_id'],
                'name': row['name'],
                'email': row['email'],
                'phone': row['phone']
            })
        duplicate_clusters.append(cluster)

print(f"\nFound {len(duplicate_clusters)} groups with multiple records for same true_id")
print("\nFirst 10 duplicate groups:")

for i, cluster in enumerate(duplicate_clusters[:10]):
    print(f"\nCluster {i+1} - true_id: {cluster['true_id']} ({cluster['count']} records):")
    for record in cluster['records']:
        print(f"  - row_id: {record['row_id']}, name: {record['name']}, email: {record['email']}, phone: {record['phone']}")

Original rows: 200

Found 100 groups with multiple records for same true_id

First 10 duplicate groups:

Cluster 1 - true_id: id-0080 (2 records):
  - row_id: row-0002, name: Chavez, Leah, email: lchavez@mail.example.org, phone: (555) 010.0080
  - row_id: row-0224, name: Laeh Chavez, email: leah.chavez2@example.net, phone: 555-010-0080

Cluster 2 - true_id: id-0013 (2 records):
  - row_id: row-0003, name: James Harris, email: nan, phone: (555) 010-0013
  - row_id: row-0190, name: Jmaes Harris, email: james.harris5@example.net, phone: 555-010-0013

Cluster 3 - true_id: id-0030 (2 records):
  - row_id: row-0004, name: Lopez, Levi, email: llopez@mail.example.org, phone: (555) 010.0030
  - row_id: row-0154, name: Lvei Lopez, email: levi.lopez1@example.net, phone: 555-010-0030

Cluster 4 - true_id: id-0011 (2 records):
  - row_id: row-0005, name: Jackson, Henry, email: hjackson@mail.example.org, phone: (555) 010.0011
  - row_id: row-0087, name: Henry Jackson, email: nan, phone: (555) 010-00

In [23]:
# true_id hidden from the dedupe process
import pandas as pd
from difflib import SequenceMatcher

# Read the CSV file
df = pd.read_csv(r'c:\Users\Dave Sisk\Repos\soft-relate-data\sample-data-messy_200.csv')

print(f"Original rows: {len(df)}")

def string_similarity(a, b):
    if pd.isna(a) or pd.isna(b):
        return 0
    return SequenceMatcher(None, str(a).lower().strip(), str(b).lower().strip()).ratio()

# Find potential duplicates by comparing names, emails, and phones (but NOT true_id)
duplicate_clusters = []

# We'll use a simple pairwise comparison for demonstration (O(n^2) for small datasets)
used = set()
for i, row1 in df.iterrows():
    if i in used:
        continue
    cluster = [row1]
    for j, row2 in df.iterrows():
        if i != j and j not in used:
            # Compare name, email, and phone for similarity
            name_sim = string_similarity(row1['name'], row2['name'])
            email_sim = string_similarity(row1['email'], row2['email'])
            phone_sim = string_similarity(row1['phone'], row2['phone'])
            # If any two fields are highly similar, consider as duplicate
            if (name_sim > 0.95 and email_sim > 0.95) or (name_sim > 0.95 and phone_sim > 0.95) or (email_sim > 0.95 and phone_sim > 0.95):
                cluster.append(row2)
                used.add(j)
    if len(cluster) > 1:
        duplicate_clusters.append(cluster)
        for r in cluster:
            used.add(r.name)

print(f"\nFound {len(duplicate_clusters)} clusters of potential duplicates (not using true_id)")
print("\nFirst 10 duplicate clusters:")

for i, cluster in enumerate(duplicate_clusters[:10]):
    print(f"\nCluster {i+1} with {len(cluster)} records:")
    for record in cluster:
        print(f"  - row_id: {record['row_id']}, name: {record['name']}, email: {record['email']}, phone: {record['phone']}")

Original rows: 200

Found 0 clusters of potential duplicates (not using true_id)

First 10 duplicate clusters:


In [34]:
import dedupe

# Enhanced duplicate detection with more aggressive matching
# while still hiding true_id from dedupe

# Fill NaN values with empty strings to avoid AttributeError
for col in ['name', 'email', 'phone', 'address']:
    df[col] = df[col].fillna('')

# Replace empty strings with a placeholder to avoid dedupe errors
for col in ['name', 'email', 'phone', 'address']:
    df[col] = df[col].replace('', '[BLANK]')

# Remove records where all dedupe fields are blank
mask = ~((df['name'] == '[BLANK]') & (df['email'] == '[BLANK]') & (df['phone'] == '[BLANK]') & (df['address'] == '[BLANK]'))
df_filtered = df[mask].copy()

# Convert dataframe to list of dicts for dedupe, excluding true_id
records_for_dedupe = df_filtered[['row_id', 'name', 'email', 'phone', 'address']].to_dict('records')

# Dedupe requires at least 2 records and at least 2 unique records for training
if len(records_for_dedupe) < 2:
    print("Not enough records to deduplicate after filtering. Skipping dedupe.")
else:
    unique_records = [tuple(rec.values()) for rec in records_for_dedupe]
    if len(set(unique_records)) < 2:
        print("All records are identical after filtering. Skipping dedupe.")
    else:
        # Prepare data for dedupe, excluding true_id
        fields_for_dedupe = [
            dedupe.variables.String('name'),
            dedupe.variables.String('email'),
            dedupe.variables.String('phone'),
            dedupe.variables.String('address')
        ]

        try:
            # Create dedupe object with more lenient thresholds
            deduper = dedupe.Dedupe(fields_for_dedupe)

            # Sample for training (use a sample to speed up)
            deduper.prepare_training({i: rec for i, rec in enumerate(records_for_dedupe)})
            deduper.train({i: rec for i, rec in enumerate(records_for_dedupe)})

            # Find matches with lower threshold for more aggressive matching
            linked_records = deduper.partition({i: rec for i, rec in enumerate(records_for_dedupe)}, threshold=0.5)

            # Print results
            print(f"Found {len(linked_records)} clusters of potential duplicates")
            print("\nFirst 10 duplicate clusters:")

            for i, cluster in enumerate(linked_records[:10]):
                print(f"\nCluster {i+1} with {len(cluster)} records:")
                for record_id in cluster:
                    record = df_filtered.iloc[record_id]
                    print(f"  - row_id: {record['row_id']}, name: {record['name']}, email: {record['email']}, phone: {record['phone']}, true_id: {record['true_id']}")
        except (ValueError, IndexError) as e:
            print(f"Dedupe encountered an error: {str(e)}")
            print("This may be due to insufficient data variation or imbalanced records.")
            print("Skipping dedupe and returning original filtered dataset.")
            print(f"Total records available: {len(df_filtered)}")

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, name)


Dedupe encountered an error: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.
This may be due to insufficient data variation or imbalanced records.
Skipping dedupe and returning original filtered dataset.
Total records available: 200
